In [3]:
# This notebook have as a point to allow us scraping job sites, to give us ability to have a DB that references 
# all available computing jobs in France.
# We should add also a country classemnet of basic salries to compare ?? TBD
# Steps:
"""
Search on :
- indeed
- monster
with keys {'informatique', 'france'} 
- Find where is job ads 
- parse and get {title, salary, location, missions, company name}
"""
#!pip install pandas
#!pip install bs4 requests lxml 
!pip install mysql-connector-python


  Using cached mysql_connector_python-8.0.22-cp36-cp36m-manylinux1_x86_64.whl (18.0 MB)
You should consider upgrading via the '/home/fbordjah/.virtualenvs/scraping_project/bin/python -m pip install --upgrade pip' command.


In [95]:
# Brouillon
#!conda install lxml requests bs4
#!conda install -y pandas
#!pip install pandas

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import mysql.connector
import time


In [76]:
# search for jobs with key 'inforamtique' and 'france' location.
ads = []
for x in range(0,10,10):
    indeed_url = 'https://fr.indeed.com/emplois?q=informatique&l=France&start={}'.format(x)
    try:
        res = requests.get(indeed_url)
        res.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    # print(res)
    # parse witrh soup
    soup = bs(res.content,"html")
    #print(type(soup))
    cards = soup.find_all({"div"},{"class":"jobsearch-SerpJobCard unifiedRow row result"})
    for card in cards:
        ad = {}
        ad["ref"] = card.attrs['id'].lstrip('\n')
        ad["job"] =  card.h2.a.text.replace('\n','')
        ad["company"] = card.find({"div"},{"class":"sjcl"}).span.text.lstrip('\n')
        ad["location"] = card.find({"div"},{"class":"sjcl"}).find({"div"},{"class":"recJobLoc"}).attrs['data-rc-loc'].lstrip('\n')
        ads.append(ad)


In [93]:
# Monster
monster_url = "https://www.monster.fr/emploi/recherche/?q=Informatique"

html = requests.get(monster_url)
soup_monster = bs(html.content, 'html')
allPostLink = soup_monster.findAll("h2", {'class', 'title'})
linksArray = []
for post in allPostLink:
    linksArray.append(post.a['href'])


In [120]:
for linkUrl in linksArray:
    try:
        res = requests.get(linkUrl)
        res.raise_for_status()
    except:
        continue
    ad_detail = bs(res.text, 'html')
    ad = {}
    ad['job'] = ad_detail.find({"h1"},{"class":"job_title"}).text
    ads.append(ad)


[{'ref': 'pj_71efcb9de9d8c3fc', 'job': 'TECHNICIEN INFORMATIQUE - PROJETS & PROXIMITÉ (F/H)', 'company': 'IZENCIA', 'location': 'Mormant-sur-Vernisson (45)'}, {'ref': 'pj_812912ff36556f1f', 'job': 'Ingénieur informatique H/F', 'company': 'IT SYSTEMES', 'location': 'Levallois-Perret (92)'}, {'ref': 'pj_5594789863775882', 'job': 'Ingénieur Coeur de réseau Voix Entreprise (centrex) H/F', 'company': 'Bouygues Telecom', 'location': 'Meudon (92)'}, {'ref': 'pj_474d15fd83ec7547', 'job': 'Technicien maintenance informatique H/F', 'company': 'Mdinformatique', 'location': 'Salaise-sur-Sanne (38)'}, {'ref': 'pj_1a52a5bbc9453ad7', 'job': 'Technicien informatique itinérant H/F 100% Hardware', 'company': 'PRESTINFO MAINTENANCE', 'location': 'Paris (75)'}, {'ref': 'pj_20cd3b9bd97ccabb', 'job': 'Technicien informatique Apple', 'company': 'Aldorande', 'location': 'Paris (75)'}, {'ref': 'p_f0751e4d23bcee98', 'job': 'Administrateur / Administratrice système informatique Junior...', 'company': 'Pôle Emplo

In [121]:
df = pd.DataFrame(ads)
df

,ref,job,company,location
0,pj_71efcb9de9d8c3fc,TECHNICIEN INFORMATIQUE - PROJETS & PROXIMITÉ ...,IZENCIA,Mormant-sur-Vernisson (45)
1,pj_812912ff36556f1f,Ingénieur informatique H/F,IT SYSTEMES,Levallois-Perret (92)
2,pj_5594789863775882,Ingénieur Coeur de réseau Voix Entreprise (cen...,Bouygues Telecom,Meudon (92)
3,pj_474d15fd83ec7547,Technicien maintenance informatique H/F,Mdinformatique,Salaise-sur-Sanne (38)
4,pj_1a52a5bbc9453ad7,Technicien informatique itinérant H/F 100% Har...,PRESTINFO MAINTENANCE,Paris (75)
5,pj_20cd3b9bd97ccabb,Technicien informatique Apple,Aldorande,Paris (75)
6,p_f0751e4d23bcee98,Administrateur / Administratrice système infor...,Pôle Emploi,Roissy-en-France (95)
7,p_2da8b14a9ae9462b,Ingénieur / Ingénieure d'étude et développemen...,Pôle Emploi,Levallois-Perret (92)
8,p_bfac5884f2682c0a,Développeur / Développeuse informatique (H/F),Pôle Emploi,Puteaux (92)
9,p_69c7e53b9a59bdad,Administrateur / Administratrice système infor...,Métropole Rouen Normandie,Rouen (76)


In [6]:
# Create database connection
mydb = mysql.connector.connect(host="localhost",
                               port="3306",
                               user="scrap",
                               password="secret",
                               database="scrap")

# Test insertions
'''mycursor = mydb.cursor()
sql = "INSERT INTO ads (job, company) VALUES (%s, %s)"
val = ("John", "Highway 21")
mycursor.execute(sql, val)
mydb.commit()
'''